In [1]:
import os, sys

PROJECT_ROOT = '/scratch/jq2uw/derm_vlms'
LLAVA_DERM_DIR = os.path.join(PROJECT_ROOT, 'llava_derm')

if LLAVA_DERM_DIR not in sys.path:
    sys.path.insert(0, LLAVA_DERM_DIR)

os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:128'

import torch
torch.cuda.empty_cache()

from utils import load_model, predict_image, parse_label

print('Loading model...')
model, processor = load_model()
print('Model loaded.')

/home/jq2uw/.local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!


Loading model...


Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.09it/s]
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Total params: 7,063,427,072
Model loaded.


In [2]:
import pandas as pd
from pathlib import Path

DATA_DIR = Path(PROJECT_ROOT) / 'data'

df = pd.read_parquet(os.path.join(PROJECT_ROOT, 'data_share', 'midas_share.parquet'))
print(f'Loaded {len(df)} rows')
print(f'y3 distribution:\n{df["y3"].value_counts()}')

def resolve_img_path(p):
    p = str(p)
    if os.path.isfile(p):
        return p
    candidate = DATA_DIR / Path(p).name
    if candidate.is_file():
        return str(candidate)
    return p

df['image_path_resolved'] = df['image_path'].apply(resolve_img_path)
n_found = df['image_path_resolved'].apply(os.path.isfile).sum()
print(f'Resolved images: {n_found}/{len(df)} found')

SEED = 42
N_PER_CLASS = 5
df_sample = df.groupby('y3', group_keys=False).apply(
    lambda g: g.sample(n=N_PER_CLASS, random_state=SEED),
).reset_index(drop=True)
print(f'\nStratified sample ({N_PER_CLASS} per class, seed={SEED}):')
print(df_sample['y3'].value_counts())
df_sample[['uid', 'y3', 'image_path_resolved']].head()

Loaded 3357 rows
y3 distribution:
y3
malignant    1391
benign       1322
other         644
Name: count, dtype: int64
Resolved images: 3357/3357 found

Stratified sample (5 per class, seed=42):
y3
benign       5
malignant    5
other        5
Name: count, dtype: int64


/scratch/jq2uw/conda_ex/tmp/ipykernel_861847/3641150031.py:25: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_sample = df.groupby('y3', group_keys=False).apply(


,uid,y3,image_path_resolved
0,1833,benign,/scratch/jq2uw/derm_vlms/data/s-prd-697891782.jpg
1,1191,benign,/scratch/jq2uw/derm_vlms/data/s-prd-593416010.jpg
2,610,benign,/scratch/jq2uw/derm_vlms/data/s-prd-639852881.jpg
3,1053,benign,/scratch/jq2uw/derm_vlms/data/s-prd-560547879.jpg
4,188,benign,/scratch/jq2uw/derm_vlms/data/s-prd-419238986.jpg


In [ ]:
from PIL import Image
from tqdm import tqdm

q_describe = "Describe the lesion in detail."
q_classify = "Is the lesion malignant or benign, or other?"
results = []

for _, row in tqdm(df_sample.iterrows(), total=len(df_sample)):
    uid = row['uid']
    try:
        image = Image.open(row['image_path_resolved']).convert('RGB')
    except Exception as e:
        print(f'[SKIP] uid={uid}: {e}')
        continue

    description = predict_image(model, processor, image, prompt=q_describe)
    classification = predict_image(model, processor, image, prompt=q_classify)

    results.append({
        'uid': uid,
        'y3': row['y3'],
        'description': description,
        'classification': classification,
    })

print(f'Collected {len(results)} predictions')

100%|█████████████████████████████████████████████████████████████████████████████████| 15/15 [00:56<00:00,  3.74s/it]

Collected 15 predictions


In [4]:
results_df = pd.DataFrame(results).set_index('uid')

results_df['pred_label'] = results_df['classification'].apply(parse_label)

print('Predicted label distribution:')
print(results_df['pred_label'].value_counts())

RESULTS_DIR = os.path.join(LLAVA_DERM_DIR, 'results')
os.makedirs(RESULTS_DIR, exist_ok=True)
out_path = os.path.join(RESULTS_DIR, 'llava_derm_predictions.csv')
results_df.to_csv(out_path)
print(f'\nSaved to {out_path}')

results_df

Predicted label distribution:
pred_label
benign       4
malignant    1
Name: count, dtype: int64

Saved to /scratch/jq2uw/derm_vlms/llava_derm/results/llava_derm_predictions.csv


,y3,description,classification,pred_label
uid,,,,
1833,benign,There is 0.55 chance that the lesion is Atypic...,The lesion is malignant.,malignant
1191,benign,There is 0.55 chance that the lesion is Hemang...,Is the lesion present in the skin of the body?,None
610,benign,There is 0.33 chance that the lesion is Lichen...,There is 0.33 chance that the lesion is Dermat...,None
1053,benign,There is 0.55 chance that the lesion is Cyst. ...,Is the lesion infected?,None
188,benign,There is 0.55 chance that the lesion is Dermat...,Is the lesion a cyst?,None
3050,malignant,There is 0.33 chance that the lesion is Hemang...,The lesion is benign.,benign
416,malignant,The lesion is a wound of the scalp. It could b...,The lesion is benign.,benign
3310,malignant,There is 0.55 chance that the lesion is Eczema...,The lesion is benign.,benign
2450,malignant,There is 0.33 chance that the lesion is Hemang...,Is the lesion in the skin or subcutaneous tiss...,None
